In [15]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

from pprint import pprint

import matplotlib.pyplot as plt
from matplotlib import font_manager, rc

from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split

import pickle
import random

In [16]:
rname = ['user', 'title', 'score']
df = pd.read_csv("rating.csv", encoding = 'utf-8-sig', sep = '\t', header = None, names = rname)
df.head()

,user,title,score
0,encore1,완득이,4.0
1,encore1,"스물아홉 생일, 1년 후 죽기로 결심했다",5.0
2,encore1,청소부 밥,5.0
3,encore1,말의 품격,5.0
4,encore1,예감은 틀리지 않는다,4.0


In [17]:
len(df['user'].unique())

35

In [18]:
def execute_labeling(column_name):
    Lec = LabelEncoder()
    df_copy = df.copy()
    df_label = pd.DataFrame(Lec.fit_transform(df_copy[column_name].values), columns = [column_name+'_label'])
    return df_label

In [19]:
df_user = execute_labeling('user')
df_title = execute_labeling('title')

print(len(df_user['user_label'].value_counts()))
print(len(df_title['title_label'].value_counts()))

df_copy = df.copy()    # copy df
df_copy = df_copy.drop(df_copy.columns[[0, 1]], axis = 1)    # delete 'user', 'title' columns
df_label = pd.concat([df_user, df_title, df_copy], axis = 1)    # 라벨링한 컬럼과 기존 데이터프레임 합치기

df_label.head(5)

35
1009


,user_label,title_label,score
0,0,699,4.0
1,0,564,5.0
2,0,866,5.0
3,0,339,5.0
4,0,680,4.0


In [20]:
movie_list = list(df['title'].unique())
movie_list = sorted(movie_list, key=str)
df_matrix = pd.DataFrame(columns = ['user'] + movie_list)
df_matrix    #결과확인

,user,12가지 인생의 법칙,13 Reasons Why,15소년 표류기,1984,1Q84,1cm 첫 번째 이야기,1cm+ 일 센티 플러스,1日1食,1리터의 눈물 마지막 편지,...,화폐전쟁,황태자비 납치사건,회계 천재가 된 홍대리 2 : 수익성 개선 편,회사가 당신에게 알려주지 않는 50가지 비밀,후니의 쉽게 쓴 시스코 네트워킹,후레자식,후불제 민주주의,흑설공주 이야기,흰,힐빌리의 노래


In [21]:
%%time

user_num = len(df_label.user_label.unique()) # 35
movie_num = len(df_label.title_label.unique()) # 1009

user_score_list = []    # df_matrix에서 'user' 컬럼을 제외한 컬럼. 즉, title 컬럼
for num in range(0, user_num):
    
    user_score_list = [0 for i in range(movie_num)]   # 값을 0으로 초기화
    df_tmp = df_label[df_label['user_label'] == num]    # 각 유저별로 묶는다
    for j in df_tmp.index:
        intL = int(df_tmp.ix[j]['title_label']-1)
        user_score_list[intL] = df_tmp.ix[j]['score']
        #print(user_score_list)
    
    df_matrix.loc[num] = [num] + user_score_list 
    #df_tmp.ix[]['score']
    #user_score_list[df_tmp.ix[j]['title_label']-1]

C:\Users\Playdata\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Playdata\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if sys.path[0] == '':


Wall time: 1.56 s


In [22]:
# 유틸리티행렬 pickle 저장
file = df_matrix
directory = open('utilitymatrix', 'wb')
pickle.dump(file, directory)
directory.close()

In [23]:
user_name = list(df['user'].unique())
#user_name = sorted(user_name, key = str)
user_name = pd.DataFrame(user_name)
user_name.columns = ['user_name']

# user_name pickle 저장
file_user = user_name
directory_user = open('user_name', 'wb')
pickle.dump(file_user, directory_user)
directory_user.close()

In [24]:
# pickle로 저장한 user_name data frame open
f = open('user_name', 'rb')
user_name = pickle.load(f)
user_name = user_name

In [25]:
# pickle로 저장한 유틸리티 행렬 DATA LOAD
f = open('utilitymatrix', 'rb')
utilitymatrix = pickle.load(f)

df2 = utilitymatrix
df2.head()


,user,12가지 인생의 법칙,13 Reasons Why,15소년 표류기,1984,1Q84,1cm 첫 번째 이야기,1cm+ 일 센티 플러스,1日1食,1리터의 눈물 마지막 편지,...,화폐전쟁,황태자비 납치사건,회계 천재가 된 홍대리 2 : 수익성 개선 편,회사가 당신에게 알려주지 않는 50가지 비밀,후니의 쉽게 쓴 시스코 네트워킹,후레자식,후불제 민주주의,흑설공주 이야기,흰,힐빌리의 노래
0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0
2,2.0,4.0,0.0,3.5,4.5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5
3,3.0,0.0,0.0,4.5,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0
4,4.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
df_concat = pd.concat([user_name, df_matrix2], axis = 1)
df_concat.head()

,user_name,user,12가지 인생의 법칙,13 Reasons Why,15소년 표류기,1984,1Q84,1cm 첫 번째 이야기,1cm+ 일 센티 플러스,1日1食,...,화폐전쟁,황태자비 납치사건,회계 천재가 된 홍대리 2 : 수익성 개선 편,회사가 당신에게 알려주지 않는 50가지 비밀,후니의 쉽게 쓴 시스코 네트워킹,후레자식,후불제 민주주의,흑설공주 이야기,흰,힐빌리의 노래
0,encore1,0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,encore2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0
2,encore3,2,4.0,0.0,3.5,4.5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5
3,encore4,3,0.0,0.0,4.5,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0
4,encore5,4,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
df_matrix2= df_matrix.copy()
df_matrix2['user']=df_matrix['user'].astype(int)
df_matrix2.head()

,user,12가지 인생의 법칙,13 Reasons Why,15소년 표류기,1984,1Q84,1cm 첫 번째 이야기,1cm+ 일 센티 플러스,1日1食,1리터의 눈물 마지막 편지,...,화폐전쟁,황태자비 납치사건,회계 천재가 된 홍대리 2 : 수익성 개선 편,회사가 당신에게 알려주지 않는 50가지 비밀,후니의 쉽게 쓴 시스코 네트워킹,후레자식,후불제 민주주의,흑설공주 이야기,흰,힐빌리의 노래
0,0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0
2,2,4.0,0.0,3.5,4.5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5
3,3,0.0,0.0,4.5,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0
4,4,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
def cosine_similarity(data_name):    
    from sklearn.metrics.pairwise import cosine_distances
    similarity = 1 - cosine_distances(data_name)    # sklearn은 정의와 반대이므로 1에서 빼준다.
    return similarity

cos_sim = cosine_similarity(df_matrix2)    # data set으로 df를 넣음
print(cos_sim.shape)
cos_sim

(35, 35)


array([[1.        , 0.13174856, 0.17916344, ..., 0.00471873, 0.0953597 ,
        0.15001084],
       [0.13174856, 1.        , 0.1247497 , ..., 0.04353675, 0.10945048,
        0.13404518],
       [0.17916344, 0.1247497 , 1.        , ..., 0.04913115, 0.14855547,
        0.12648642],
       ...,
       [0.00471873, 0.04353675, 0.04913115, ..., 1.        , 0.49062219,
        0.53917641],
       [0.0953597 , 0.10945048, 0.14855547, ..., 0.49062219, 1.        ,
        0.63113405],
       [0.15001084, 0.13404518, 0.12648642, ..., 0.53917641, 0.63113405,
        1.        ]])

In [29]:
class Basic(object):
    
    def __init__(self, user_name, neigh_num):
        self.user_name = user_name
        self.neigh_num = neigh_num
    
    """유저의 이름이 들어오면 유저 번호로 변환"""
    def convert_user_num(self):
        
        user_name_list = list(df_concat['user_name'].unique())
        
        for num in range(len(user_name_list)):
            if user_name_list[num] == self.user_name:
                user_num = num
        
        return user_num
    
    
    """target 유저와 유사한 유저 K명을 찾고, cosine 유사도를 이용하여 거리를 구한다"""
    def find_near_neighbor(self):
    
        from sklearn.neighbors import NearestNeighbors
        
        user_num = Basic.convert_user_num(self)
            
        KNN = NearestNeighbors(n_neighbors = self.neigh_num, metric = 'cosine')    # n_neighbors에는 본인이 포함되기 때문에 +1을 해준다.
        KNN.fit(df_matrix2)    # data set은 utility matrix인 df를 사용
        similar_distance, similar_users = KNN.kneighbors(df_matrix2)
    
        similars = {}    #유사한 유저와 거리를 dict형식으로 저장
    
        # 유사한 유저
        similar_users = similar_users[user_num][1:]
        similars['sim_users'] = list(similar_users)
    
        # 유사한 유저들과의 거리
        similar_distance = similar_distance[user_num][1:]
        similars['sim_distance'] = similar_distance
    
        return similars
    
    
    """target유저 + 유사한 유저 K명으로 이루어진 새로운 data frame 형성하고, narray 형식으로 반환"""
    def near_neighbors_narray(self):
    
        similars = Basic.find_near_neighbor(self)
        similiar_users_list = similars['sim_users']
        similiar_distances = similars['sim_distance']

        columns = list(df_matrix2.columns)
        new_df = pd.DataFrame(columns = columns)
    
        for i in range(len(similiar_users_list)):
        
            def concat_row(i):
                neighbor_df = df_matrix2[df_matrix2['user'] == similiar_users_list[i]]
                return neighbor_df
        
            neighbor_df = pd.concat([new_df, concat_row(i)])
            new_df = neighbor_df
            
        narray = new_df.values
        narray = narray[:, 1:]
        
        return narray

문제 
- 예상평점이 float이 아니라 int로 나옴
- 책 추천뿐만 아니라 예상평점까지

In [30]:
user1 = Basic('encore34', 4)
#print(user1.convert_user_num())
#print(user1.find_near_neighbor())
print(user1.near_neighbors_narray())

[[0.0 0.0 0.0 ... 0.0 0.0 0.0]
 [0.0 0.0 0.0 ... 0.0 0.0 0.0]
 [0.0 0.0 0.0 ... 0.0 0.0 0.0]]


In [44]:
print(user1.find_near_neighbor())

{'sim_users': [29, 34, 23], 'sim_distance': array([0.34809413, 0.36886595, 0.37556667])}


In [31]:
class Calculation_rating(Basic):
    
    def __init__(self, user_name, neigh_num):
        Basic.__init__(self, user_name, neigh_num)
        
    """가중평균 값으로 아이템에 대한 target 유저의 평점을 예측"""
    def predict_rating(self):
        
        narray = Basic.near_neighbors_narray(self)    #narray 받음
        similars = Basic.find_near_neighbor(self)
        
        similiar_distances = similars['sim_distance']
    
        rating_list = []    # 가중평균값을 담는 리스트
    
        # 범위 0 ~ K-1
        for col_num in range(narray.shape[1]):
        
            sum = 0
            rating = 0
            for i in range(1, len(narray[:, col_num])):
                 sum += float(narray[:, col_num][i]) * float(similiar_distances[i])
            rating = sum/similiar_distances.sum()
        
            if rating < 0:
                rating = 0    # 만약 가중평균값이 0보다 작으면 0점으로 함
            elif rating > 10:
                rating = 10    # 만약 가중평균값이 10보다 크면 10점으로 함
            else:
                rating = int(rating)    # 평점은 정수형
    
            rating_list.append(rating)

        return rating_list
        
    
    """target 유저의 평점을 리스트로 변환하는 함수"""
    def original_rating(self):
        
        user_num = Basic.convert_user_num(self)
    
        # target 유저의 평점을 narray로 변환
        target_df = df_matrix2[df_matrix2['user'] == user_num]
        target_narray = target_df.values
        target_narray = target_narray[:, 1:]    # user column 삭제
    
        #narray로 변환된 target 유저의 평점을 리스트로 변환
        target_user_rating_list = []
        for i in range(target_narray.shape[1]):
            raw_rating = int(target_narray[0][i])
            target_user_rating_list.append(raw_rating)
    
        return target_user_rating_list

In [32]:
user_encore11 = Calculation_rating('encore1', 6)

In [33]:
user_encore11

In [34]:
print(user_encore11.original_rating()[175:190])

[0, 4, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0]


In [45]:
print(user_encore11.predict_rating()[175:190])

[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]


In [35]:
df_matrix2.loc[0][175:190]

나무                 0.0
나미야 잡화점의 기적        0.0
나의 라임 오렌지 나무       4.0
나의 라임 오렌지나무        0.0
나의 문화유산답사기         0.0
나의 한국현대사           3.5
난장이가 쏘아올린 작은 공     0.0
날개                 0.0
날개 : 이상 소설전집       0.0
낢이 사는 이야기          3.0
남과 여 1             0.0
남쪽으로 튀어!           0.0
남한산성               0.0
낭만적 연애와 그 후의 일상    0.0
낮에 뜨는 달            0.0
Name: 0, dtype: float64

In [36]:
user_encore11 = Calculation_rating('encore33', 6)

In [37]:
aa= user_encore11.predict_rating()
bb = df_matrix.columns

In [38]:
bbb = pd.DataFrame(bb)
aaa = pd.DataFrame(aa)

In [39]:
bbbb = bbb[1:]

In [40]:
df.head()

,user,title,score
0,encore1,완득이,4.0
1,encore1,"스물아홉 생일, 1년 후 죽기로 결심했다",5.0
2,encore1,청소부 밥,5.0
3,encore1,말의 품격,5.0
4,encore1,예감은 틀리지 않는다,4.0


In [41]:
np.where(df["user"] == 'encore33')

(array([1920, 1921, 1922, 1923, 1924, 1925, 1926, 1927, 1928, 1929, 1930,
        1931, 1932, 1933, 1934, 1935, 1936, 1937, 1938, 1939, 1940, 1941,
        1942, 1943, 1944, 1945, 1946, 1947, 1948, 1949, 1950, 1951, 1952,
        1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963,
        1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974,
        1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985,
        1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996,
        1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007,
        2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018],
       dtype=int64),)

In [42]:
df[1920: 2019]

,user,title,score
1920,encore33,그리스인 조르바 - 전2권 (한글판 + 영문판),4.0
1921,encore33,남쪽으로 튀어!,3.0
1922,encore33,도가니,2.5
1923,encore33,연금술사,2.0
1924,encore33,왜 나는 너를 사랑하는가,1.0
1925,encore33,경제학 콘서트,3.0
1926,encore33,괴짜경제학,3.0
1927,encore33,구글 신은 모든 것을 알고 있다,3.0
1928,encore33,토지,4.0
1929,encore33,버리고 갈 것만 남아서 참 홀가분하다,3.0


In [43]:
df['user']

0        encore1
1        encore1
2        encore1
3        encore1
4        encore1
5        encore1
6        encore1
7        encore1
8        encore1
9        encore1
10       encore1
11       encore1
12       encore1
13       encore1
14       encore1
15       encore1
16       encore1
17       encore1
18       encore1
19       encore1
20       encore1
21       encore1
22       encore1
23       encore1
24       encore1
25       encore1
26       encore1
27       encore1
28       encore1
29       encore1
          ...   
2190    encore35
2191    encore35
2192    encore35
2193    encore35
2194    encore35
2195    encore35
2196    encore35
2197    encore35
2198    encore35
2199    encore35
2200    encore35
2201    encore35
2202    encore35
2203    encore35
2204    encore35
2205    encore35
2206    encore35
2207    encore35
2208    encore35
2209    encore35
2210    encore35
2211    encore35
2212    encore35
2213    encore35
2214    encore35
2215    encore35
2216    encore35
2217    encore

In [231]:
b1 = bbbb.reset_index()
b2 = pd.DataFrame(b1[0])
d2 = pd.concat([b2, aaa], axis = 1)
d2

,0,0
0,12가지 인생의 법칙,0
1,13 Reasons Why,0
2,15소년 표류기,0
3,1984,0
4,1Q84,0
5,1cm 첫 번째 이야기,0
6,1cm+ 일 센티 플러스,0
7,1日1食,0
8,1리터의 눈물 마지막 편지,0
9,1만 시간 동안의 남미,0


In [218]:
print(bbbb.tail(10))


                              0
1000                       화폐전쟁
1001                  황태자비 납치사건
1002  회계 천재가 된 홍대리 2 : 수익성 개선 편
1003   회사가 당신에게 알려주지 않는 50가지 비밀
1004          후니의 쉽게 쓴 시스코 네트워킹
1005                       후레자식
1006                   후불제 민주주의
1007                   흑설공주 이야기
1008                          흰
1009                    힐빌리의 노래


In [213]:
d = pd.concat([bbbb, aaa], axis = 1, ignore_index=True)
d

,0,1
0,NaN,0.0
1,12가지 인생의 법칙,0.0
2,13 Reasons Why,1.0
3,15소년 표류기,0.0
4,1984,1.0
5,1Q84,0.0
6,1cm 첫 번째 이야기,0.0
7,1cm+ 일 센티 플러스,0.0
8,1日1食,0.0
9,1리터의 눈물 마지막 편지,0.0


In [209]:
aaa

,0
0,0
1,0
2,1
3,0
4,1
5,0
6,0
7,0
8,0
9,0


In [184]:
# MSE를 이용한 성능평가 함수
def evaluation(neigh_num):
    
    from sklearn.metrics import mean_squared_error
    
    neigh_num = neigh_num
    rmse = 0
    user_name_list = list(df_concat['user_name'].unique())
    
    for user_name in user_name_list:
        
        cal = Calculation_rating(user_name, neigh_num)
        predict_list = cal.predict_rating()
        original_list = cal.original_rating()
        rmse_onebyone = mean_squared_error(original_list, predict_list)
        
        rmse += rmse_onebyone
        
    rmse = rmse/len(user_name_list)

    return print('이웃의 수가 {}일때, MSE 값은 {}입니다.'.format(neigh_num-1, rmse))

In [186]:
%%time
neigh_num = [10, 20, 30]    # 이웃의 수가 20명, 40명, 60명이 됨
for num in neigh_num:
    evaluation(num)

이웃의 수가 9일때, MSE 값은 0.8292510264760017입니다.
이웃의 수가 19일때, MSE 값은 0.8350276086648731입니다.
이웃의 수가 29일때, MSE 값은 0.8407475576950302입니다.
Wall time: 13.8 s


In [187]:
%%time
neigh_num = [6, 8, 11]    # 이웃의 수가 5명, 7명, 10명이 됨
for num in neigh_num:
    evaluation(num)

이웃의 수가 5일때, MSE 값은 0.8471754212091177입니다.
이웃의 수가 7일때, MSE 값은 0.8314314030865072입니다.
이웃의 수가 10일때, MSE 값은 0.8267025343338524입니다.
Wall time: 9.79 s


In [46]:
class UBCF(Calculation_rating):
    
    def __init__ (self, user_name, neigh_num):
        Basic.__init__(self, user_name, neigh_num)
        Calculation_rating.__init__(self, user_name, neigh_num)        
        
    def recommen_movie_list(self):
        
        user_num = Basic.convert_user_num(self)
        predict_list = Calculation_rating.predict_rating(self)
        original_list = Calculation_rating.original_rating(self)
        all_movie_list = list(df_matrix2.columns)[1:]    # 전체 영화 리스트
        
        """
        target 유저가 이미 평가했던 영화 외의 영화를 추천받기 위해
        target 유저의 평점이 0이면, 가중평균값을 넣고 그렇지 않으면 0을 넣는다
        """
        temp_list = []
        for i in range(len(predict_list)):
            if int(original_list[i]) != 0:
                temp_list.append(0)
            else:
                temp_list.append(int(predict_list[i]))
        
        # 4점 이상인 영화들만 선택하여 index를 추천 리스트에 담는다.
        recommen_list_index =[]
        for i in range(len(temp_list)):
            if temp_list[i] >= 5:
                recommen_list_index.append(i)
        
        # recommen_list_index로 부터 영화 제목을 str로 저장
        recommen_list_str = []
        for i in recommen_list_index:
            recommen_list_str.append(all_movie_list[i])
        
        """전체 영화 리스트에서 target 유저가 이미 평가한 영화를 제거"""
        already_rating_movie_num = [i for i in range(len(temp_list)) if temp_list[i] == 0]
        user_movie_list = [all_movie_list[i] for i in range(len(all_movie_list)) if i not in already_rating_movie_num]

        
        final_dict = {}
        final_dict['by_rating'] = recommen_list_str
        final_dict['by_delete'] = user_movie_list
        
        return final_dict
    
    
    """영화 추천 실행 함수"""
    def recommendation(self):
        
        user_number = Basic.convert_user_num(self)
        movie_dict = UBCF.recommen_movie_list(self)
        
        by_rating_list = movie_dict['by_rating']
        by_delete_list = movie_dict['by_delete']
        
        # 만약 추천 리스트가 3개 이상이라면 가중평균 리스트에서 3편의 영화를 추천리스트에서 추천
        # 그렇지 않으면 유저가 이미 평가했던 영화를 제외한 영화 리스트에서 추천
        if len(by_rating_list) >= 3:
            recommendation_selection = random.sample(by_rating_list, 3)
        else:
            if len(by_delete_list) >= 3 :
                recommendation_selection = random.sample(by_delete_list, 3)
            else:
                recommendation_selection = random.sample(by_delete_list, 1)
            
        user_name = df_concat['user_name'][user_number]    # user number를 user의 아이디로 변경
    
        print('{}님을 위한 추천 영화 입니다.'.format(user_name))
        return print(recommendation_selection)

In [47]:
recommendation1 = UBCF('encore33', 10)
recommendation1.recommendation()

encore33님을 위한 추천 영화 입니다.
['해리 포터와 죽음의 성물', '설민석의 조선왕조실록', '나목']


In [196]:
df_concat.loc[32]

user_name                           encore33
user                                      32
12가지 인생의 법칙                                0
13 Reasons Why                             0
15소년 표류기                                   0
1984                                       0
1Q84                                       0
1cm 첫 번째 이야기                               0
1cm+ 일 센티 플러스                              0
1日1食                                       0
1리터의 눈물 마지막 편지                             0
1만 시간 동안의 남미                               0
1만 시간의 법칙                                  0
1만권 독서법                                    0
1시간에 1권 퀀텀 독서법                             0
1인 가구 돈 관리                                 0
2012 마음의 소리 Ver.1                          0
2013 마음의 소리 Ver.1                          0
2015 시나공 정보처리산업기사 필기                       0
2017 대한민국 트렌드                              0
2017 시나공 정보처리기사 필기 + 별책부록(기출문제집)           0
2018 인구 절벽이 온다                             0
20대, 공부에 미